**IBM Data Science Capstone Week 3**

**Install wikipedia**

In [1]:
!conda install -c conda-forge wikipedia

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - wikipedia


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    wikipedia-1.4.0            |             py_2          13 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

    python_abi:      3.6-1_cp36m       conda-forge
    wikipedia:       1.4.0-py_2        conda-forge

The following packages will be UPDATED:

   

Lets import necessary libraries 

In [2]:
import pandas as pd
import wikipedia as wp
import numpy as np

In [8]:
#Get the html source
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]

Lets see our data frame

In [9]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [32]:
print('the shape of the dataframe: ', df.shape)

the shape of the dataframe:  (103, 3)


**1. Remove 'Not assigned' from Borough column**

In [11]:
df = df[df['Borough'] != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [12]:
df.shape

(210, 3)

**2. Combine rows with the same postcode**

In [17]:
df= df.groupby(['Postcode', 'Borough'], sort = False).agg(', '.join)
df.reset_index(inplace = True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


**Check whether we have "Not assigned" Neighborhoods**

In [25]:
df[df['Neighbourhood'] == 'Not assigned']

,Postcode,Borough,Neighbourhood


In [26]:
df[df['Borough'] == 'Not assigned']

,Postcode,Borough,Neighbourhood


In [33]:
print('the shape of the dataframe: ', df.shape)

the shape of the dataframe:  (103, 3)


**2nd PART**

**import dataframe of longitute and latitute**

In [54]:
df_lat_log = pd.read_csv('https://cocl.us/Geospatial_data')
df_lat_log.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [55]:
print('the shape of the df_lat_log: ', df_lat_log.shape)

the shape of the df_lat_log:  (103, 3)


**change the column name 'Postal Code' into 'Postcode'**

In [58]:
df_lat_log.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [61]:
df_lat_log.rename(columns={"Postal Code": "Postcode"}, inplace = True)
df_lat_log.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**merge this 2 dataframes using the column 'Postcode'**

In [64]:
df_new = pd.merge(df, df_lat_log, on = 'Postcode')
df_new.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M9W,Etobicoke,Northwest,43.706748,-79.594054
1,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
2,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
3,M9P,Etobicoke,Westmount,43.696319,-79.532242
4,M9N,York,Weston,43.706876,-79.518188
5,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
6,M9L,North York,Humber Summit,43.756303,-79.565963
7,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201
8,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724
9,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242


In [63]:
df_new.shape

(103, 5)

**From now on, I am going to work with only boroughs that contain the word Toronto**

In [68]:
df2 = df_new[df_new.Borough.str.contains('Toronto',regex=False)]